Notebook à exécuter depuis wsl

In [1]:
import yaml

# Chemin du fichier MLmodel dans WSL
model_path = "/mnt/c/Users/cecil/Documents/oc_aiep7/models/Azure"

# Charger le fichier MLmodel
with open(f"{model_path}/MLmodel", 'r') as f:
    model_meta = yaml.safe_load(f)

print(model_meta)


{'artifact_path': 'outputs/mlflow-model', 'flavors': {'python_function': {'env': {'conda': 'conda.yaml', 'virtualenv': 'python_env.yaml'}, 'loader_module': 'mlflow.sklearn', 'model_path': 'model.pkl', 'predict_fn': 'predict', 'python_version': '3.9.20'}, 'sklearn': {'code': None, 'pickled_model': 'model.pkl', 'serialization_format': 'pickle', 'sklearn_version': '1.5.1'}}, 'metadata': {'azureml.base_image': 'mcr.microsoft.com/azureml/curated/ai-ml-automl:7', 'azureml.engine': 'automl'}, 'mlflow_version': '2.9.2', 'model_size_bytes': 11743049, 'model_uuid': '4f2597a615a847a4a74544353bbb83fa', 'run_id': 'sweet_bridge_1rpqblh627_256', 'signature': {'inputs': '[{"type": "string", "name": "Column2"}, {"type": "string", "name": "text"}]', 'outputs': '[{"type": "tensor", "tensor-spec": {"dtype": "int64", "shape": [-1]}}]', 'params': None}, 'utc_time_created': '2024-10-24 01:06:45.421770'}


In [3]:
# Obtenir les dépendances du modèle
# dependencies_file = mlflow.pyfunc.get_model_dependencies(model_path)
# print(f"Dependencies file created at: {dependencies_file}")

2024/11/03 22:51:25 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /mnt/c/Users/cecil/Documents/oc_aiep7/models/Azure/requirements.txt'.


Dependencies file created at: /mnt/c/Users/cecil/Documents/oc_aiep7/models/Azure/requirements.txt


In [5]:
import mlflow.pyfunc

# Charger le modèle avec mlflow
model = mlflow.pyfunc.load_model(model_path)

In [6]:
# Classe de modèle personnalisé
class MyModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model

    def predict(self, context, model_input):
        # Utiliser la méthode de prédiction du modèle chargé ici
        return self.model.predict(model_input)

In [7]:
# Définir l'URI de suivi MLflow
mlflow.set_tracking_uri("http://localhost:5000")  # Changez l'URI si nécessaire

# Terminer tout run MLflow en cours
if mlflow.active_run():
    mlflow.end_run()

# Démarrer un nouveau run dans l'expérience existante "modèle_classique"
mlflow.set_experiment("approche_classique")

<Experiment: artifact_location='mlflow-artifacts:/374735653194037029', creation_time=1729108772155, experiment_id='374735653194037029', last_update_time=1729108772155, lifecycle_stage='active', name='approche_classique', tags={}>

In [8]:
import mlflow
import mlflow.pyfunc
import os
import yaml

# Classe de modèle personnalisé
class MyModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model

    def predict(self, context, model_input):
        # Utiliser la méthode de prédiction du modèle chargé ici
        return self.model.predict(model_input)

# Définir le chemin du modèle (le dossier contenant model.pkl et MLmodel)
model_path = "/mnt/c/Users/cecil/Documents/oc_aiep7/models/Azure"

# Charger le modèle avec MLflow (ex. modèle scikit-learn ou autre)
model = mlflow.pyfunc.load_model(model_path)

# Enregistrer le modèle dans MLflow avec une classe PythonModel
mlflow.set_tracking_uri("http://localhost:5000")  # Changez l'URI si nécessaire

with mlflow.start_run(run_name="AutoML"):
    # Envelopper le modèle avec votre classe personnalisée MyModelWrapper
    wrapped_model = MyModelWrapper(model)

    # Logguer le modèle dans MLFlow
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=wrapped_model,  # Passer l'instance de la classe PythonModel ici
    )


2024/11/03 23:01:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpwapcik8h/model, flavor: python_function). Fall back to return ['cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2024/11/03 23:01:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/03 23:01:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run AutoML at: http://localhost:5000/#/experiments/374735653194037029/runs/77e71ba904b64566bed9099f0fa2be0c.
2024/11/03 23:01:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/374735653194037029.


Le modèle ne contient pas de input_example, utilisation du code proposé dans artifacts <br>
Problème: en utilisant le même dataframe que dans AutoML le schéma de données ne correspond pas !

In [9]:
import pandas as pd
example_df = pd.read_csv("./data/test_data_autoML.csv")
example_df.head(5)

,text,target
0,So my flight was supposed to leave at 10:45 th...,1
1,Right! Im checking out the hotel and heading ...,0
2,packing baby... don't wanna keep the plane wai...,0
3,@AmberHarmon My need for the latest &amp; grea...,1
4,is gettn ready to board this plane to LA...i m...,0


In [10]:
import mlflow.pyfunc
import pandas as pd

# Modèle URI
model_uri = 'runs:/22e60a8114ae48d788b585e66ad3bb58/model'

# Charger le modèle
model = mlflow.pyfunc.load_model(model_uri)

# Exemple d'entrée - un DataFrame avec les colonnes 'text' et 'target'
input_example = example_df

# Essayer une prédiction
try:
    predictions = model.predict(input_example)
    print("Prédictions : ", predictions)
except Exception as e:
    print("Erreur lors de la validation de l'exemple d'entrée : ", str(e))


2024/11/03 23:01:23 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.17.0, required: mlflow==2.9.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Erreur lors de la validation de l'exemple d'entrée :  'ColSpec' object has no attribute '_required'


In [12]:
import mlflow
logged_model = 'runs:/22e60a8114ae48d788b585e66ad3bb58/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(example_df)

2024/11/03 23:03:28 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.17.0, required: mlflow==2.9.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/cecile_2301/miniconda3/envs/AutoML-env/lib/python3.9/site-packages/mlflow/pyfunc/__init__.py", line 668, in _validate_prediction_input
    _warn_potentially_incompatible_py_version_if_necessary(model_py_version=model_py_version)
  File "/home/cecile_2301/miniconda3/envs/AutoML-env/lib/python3.9/site-packages/mlflow/models/utils.py", line 1149, in _enforce_schema
  File "/home/cecile_2301/miniconda3/envs/AutoML-env/lib/python3.9/site-packages/mlflow/types/schema.py", line 990, in required_input_names
  File "/home/cecile_2301/miniconda3/envs/AutoML-env/lib/python3.9/site-packages/mlflow/types/schema.py", line 990, in <listcomp>
  File "/home/cecile_2301/miniconda3/envs/AutoML-env/lib/python3.9/site-packages/mlflow/types/schema.py", line 729, in required
    param_names = [param_spec.name for param_spec in params]
AttributeError: 'ColSpec' object has no attribute '_required'

During handling of the above exception, another exception occurr